In [9]:
!pip install requests psycopg2-binary matplotlib


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [18]:
import pandas as pd
import requests
import time
from datetime import datetime, timedelta

In [11]:
CITY = "Lagos"
LAT = 6.5244
LON = 3.3792
API_KEY = "e9361240b9815bfe70f317a23ddff4c"

In [12]:
## Current Weather
WEATHER_URL = f"https://api.openweathermap.org/data/2.5/weather?lat={LAT}&lon={LON}&appid={API_KEY}&units=metric"

## Air Pollution (AQI)
AQI_URL = f"https://api.openweathermap.org/data/2.5/air_pollution?lat={LAT}&lon={LON}&appid={API_KEY}"

In [22]:
def fetch_weather():
    """Fetch current weather data including city, temperature, humidity, wind, and weather description."""
    try:
        response = requests.get(WEATHER_URL)
        response.raise_for_status()  # To Raise exception for bad HTTP status codes
        res = response.json()

        required_keys = ["name", "main", "wind", "weather"] # specify required keys to check if exists in the response
        if all(key in res for key in required_keys) and \
           "temp" in res["main"] and "humidity" in res["main"]:

            return {
                "city": res["name"],
                "temp_c": res["main"]["temp"],
                "humidity": res["main"]["humidity"],
                "wind": res["wind"], 
                "weather": res["weather"],
                "timestamp": datetime.now().isoformat()
            }
        else:
            return {"error": "Missing expected keys in the weather response."}

    except requests.exceptions.RequestException as e:
        return {"error": f"HTTP request failed: {str(e)}"}
    except ValueError:
        return {"error": "Failed to parse JSON from response."}
    except Exception as e:
        return {"error": f"Unexpected error: {str(e)}"}

weather = fetch_weather()
print(weather)

{'city': 'Lagos', 'temp_c': 24.12, 'humidity': 94, 'wind': {'speed': 2.06, 'deg': 230}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03d'}], 'timestamp': '2025-08-24T08:49:32.227856'}


In [ ]:
### AQI stands for Air Quality Index = 
### An AQI of 50 or below (green) is generally considered good, while an AQI above 100 indicates unhealthy conditions

In [20]:
def fetch_aqi():
    """Fetch current air quality index + pollutants"""
    res = requests.get(AQI_URL).json()
    data = res["list"][0]
    
    return {
        "aqi": data["main"]["aqi"],   # 1=Good, 2=Fair, 3=Moderate, 4=Poor, 5=Very Poor
        "pm2_5": data["components"]["pm2_5"],
        "pm10": data["components"]["pm10"],
        "o3": data["components"]["o3"],
        "co": data["components"]["co"],
        "no": data["components"]["no"],
        "no2": data["components"]["no2"],
        "so2": data["components"]["so2"],
        "nh3": data["components"]["nh3"],
        "dt": data["dt"],
        "timestamp": datetime.now().isoformat()
    }

aqi = fetch_aqi()
print(aqi)

{'aqi': 1, 'pm2_5': 3.94, 'pm10': 9.28, 'o3': 53.56, 'co': 143.32, 'no': 0, 'no2': 0.08, 'so2': 0.26, 'nh3': 0.04, 'dt': 1756021715, 'timestamp': '2025-08-24T08:48:35.420369'}


In [15]:
import psycopg2

# AQI Category Mapping
AQI_CATEGORIES = {
    1: "Good",
    2: "Fair",
    3: "Moderate",
    4: "Poor",
    5: "Very Poor"
}

# PostgreSQL connection config
DB_CONFIG = {
    'host': 'localhost',
    'port': '5432',
    'dbname': 'air_quality_db',
    'user': 'postgres',
    'password': 'Password#123'
}

"""CREATE DATABASE air_quality_db
    WITH
    OWNER = postgres
    ENCODING = 'UTF8'
    LOCALE_PROVIDER = 'libc'
    CONNECTION LIMIT = -1
    IS_TEMPLATE = False;

COMMENT ON DATABASE air_quality_db
    IS 'Real-time Air Quality & Weather Monitoring Project';"""

# Create tables if they don't exist
def create_tables():
    create_sensor_table = """
    CREATE TABLE IF NOT EXISTS sensor_readings (
        id SERIAL PRIMARY KEY,
        timestamp TIMESTAMP,
        temperature FLOAT,
        humidity FLOAT,
        weather TEXT,
        aqi_index INT,
        pm2_5 FLOAT,
        pm10 FLOAT,
        no2 FLOAT,
        o3 FLOAT,
        co FLOAT,
        aqi_category TEXT,
        aqi_change FLOAT,
        spike_detected BOOLEAN
    );
    """

    create_alert_table = """
    CREATE TABLE IF NOT EXISTS aqi_alerts (
        id SERIAL PRIMARY KEY,
        timestamp TIMESTAMP,
        aqi_index INT,
        aqi_change FLOAT,
        aqi_category TEXT
    );
    """

    with psycopg2.connect(**DB_CONFIG) as conn:
        with conn.cursor() as cur:
            cur.execute(create_sensor_table)
            cur.execute(create_alert_table)
        conn.commit()

    print("✅ Tables with primary keys created (if not exist)")

# Transform and detect spikes
def transform_data(raw_data):
    if not isinstance(raw_data, list):
        raise ValueError("raw_data must be a list of dictionaries")
        
    df = pd.DataFrame(raw_data)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['aqi_category'] = df['aqi_index'].map(AQI_CATEGORIES)
    df = df.sort_values(by='timestamp')
    df['aqi_change'] = df['aqi_index'].diff().fillna(0)
    df['spike_detected'] = df['aqi_change'].abs() >= 2
    alerts_df = df[df['spike_detected']][['timestamp', 'aqi_index', 'aqi_change', 'aqi_category']]

    # Show preview and shape
    print("📋 Transformed Sensor Data (first 5 rows):\n", df.head())
    print("📐 Shape of Sensor Data:", df.shape)
    print("🚨 Detected Alerts (if any):\n", alerts_df)
    
    return df, alerts_df

# Save data using psycopg2
def save_to_postgresql(sensor_df, alert_df):
    insert_sensor_sql = """
    INSERT INTO sensor_readings (
        timestamp, temperature, humidity, weather, aqi_index,
        pm2_5, pm10, no2, o3, co,
        aqi_category, aqi_change, spike_detected
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
    """

    insert_alert_sql = """
    INSERT INTO aqi_alerts (
        timestamp, aqi_index, aqi_change, aqi_category
    ) VALUES (%s, %s, %s, %s);
    """

    with psycopg2.connect(**DB_CONFIG) as conn:
        with conn.cursor() as cur:
            # Insert sensor readings
            for _, row in sensor_df.iterrows():
                cur.execute(insert_sensor_sql, (
                    row['timestamp'], row['temperature'], row['humidity'], row['weather'],
                    row['aqi_index'], row['pm2_5'], row['pm10'], row['no2'],
                    row['o3'], row['co'], row['aqi_category'],
                    row['aqi_change'], row['spike_detected']
                ))

            # Insert alerts
            for _, row in alert_df.iterrows():
                cur.execute(insert_alert_sql, (
                    row['timestamp'], row['aqi_index'],
                    row['aqi_change'], row['aqi_category']
                ))

        conn.commit()
    print("✅ Data saved to PostgreSQL")    

In [16]:
### Get Sensor Reading

def get_sensor_reading():
    """Fetch and combine weather + AQI data into a single dictionary suitable for transformation."""
    weather = fetch_weather()
    aqi = fetch_aqi()

    # Check for error in weather response
    if "error" in weather:
        print(f"⚠️ Weather fetch error: {weather['error']}")
        return None

    # Check for error in AQI response
    if "error" in aqi:
        print(f"⚠️ AQI fetch error: {aqi['error']}")
        return None

    # Build unified reading
    sensor_data = {
        "timestamp": weather["timestamp"],  # you can choose aqi["timestamp"] too
        "temperature": weather["temp_c"],
        "humidity": weather["humidity"],
        "weather": weather["weather"][0]["description"] if weather["weather"] else None,
        "aqi_index": aqi["aqi"],
        "pm2_5": aqi["pm2_5"],
        "pm10": aqi["pm10"],
        "no2": aqi["no2"],
        "o3": aqi["o3"],
        "co": aqi["co"]
    }

    return sensor_data


In [7]:
## Simulate Data from Original one Fetched
import random

def generate_synthetic_data(base_reading, count=1000):
    """Generate a dataset by duplicating and modifying a base sensor reading."""
    dataset = []
    base_time = datetime.utcnow()

    for i in range(count):
        reading = base_reading.copy()
        reading['timestamp'] = (base_time - timedelta(hours=i)).isoformat()

        # Add small variation to simulate real changes
        reading['temperature'] += random.uniform(-3, 3)
        reading['humidity'] += random.uniform(-10, 10)
        reading['aqi_index'] = min(max(1, reading['aqi_index'] + random.randint(-1, 1)), 5)
        reading['pm2_5'] += random.uniform(-5, 5)
        reading['pm10'] += random.uniform(-10, 10)
        reading['no2'] += random.uniform(-2, 2)
        reading['o3'] += random.uniform(-5, 5)
        reading['co'] += random.uniform(-15, 15)

        dataset.append(reading)

    return dataset


In [23]:
## Get Reading and save to Postgres

create_tables()

base = get_sensor_reading()
if base:
    data = generate_synthetic_data(base, count=1000)
    sensor_df, alert_df = transform_data(data)
    save_to_postgresql(sensor_df, alert_df)
else:
    print("❌ Failed to collect valid sensor data.")

✅ Tables with primary keys created (if not exist)


/var/folders/cw/w453z8v928g5ctbv93r__t0w0000gn/T/ipykernel_67533/807651284.py:8: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  base_time = datetime.utcnow()


📋 Transformed Sensor Data (first 5 rows):
                      timestamp  temperature    humidity           weather  \
999 2025-07-13 16:49:44.182773    25.163146   98.542668  scattered clouds   
998 2025-07-13 17:49:44.182773    21.357205  103.895912  scattered clouds   
997 2025-07-13 18:49:44.182773    22.319029   95.468070  scattered clouds   
996 2025-07-13 19:49:44.182773    24.933970   97.221680  scattered clouds   
995 2025-07-13 20:49:44.182773    24.873949   95.371920  scattered clouds   

     aqi_index     pm2_5       pm10       no2         o3          co  \
999          1  2.251002   8.189890 -1.636048  57.707013  143.339950   
998          2  6.680061   7.359847  2.000525  48.589822  139.925494   
997          1  7.787768  13.571453 -1.002684  58.301940  146.310454   
996          1  3.692194   0.376996  0.803268  56.238674  137.488427   
995          2  8.142943  17.939088 -0.545108  50.407153  158.061727   

    aqi_category  aqi_change  spike_detected  
999         Go